# Festival Playlists

In [1]:
import numpy as np
import pandas as pd
import requests
import json
import spotipy
from IPython.display import display

1. Use the Songkick API to get all the bands playing the festival
2. Use the Setlist.FM API to get the setlists
3. Use the Spotify API to create the playlists and add all the songs

### Set API credentials

In [2]:
with open('../auth.json') as f:
    credentials = json.load(f)

setlistfm_api_key = credentials['setlistfm']['api_key']
spotify_client_id = credentials['spotify']['client_id']
spotify_client_secret = credentials['spotify']['client_secret']

### Setlist FM

#### Plan of action
1. Given a lineup (list of band names), get their Musicbrainz identifiers (`mbid`) via `https://api.setlist.fm/rest/1.0/search/artists`
2. Retrieve the setlists for each artist using their `mbid` via `https://api.setlist.fm/rest/1.0/artist/{artist_mbid}/setlists
`

In [3]:
lineup = pd.read_csv(
    '../data/2022.csv', header=None, names=['band'], encoding="ISO-8859-1"
)['band'].values

In [4]:
len(lineup)

157

In [5]:
lineup

array(['Dua Lipa', 'Gorillaz', 'Interpol', 'Tyler, The Creator',
       'Ashnikko', 'Bad Gyal', 'Bicep', 'Big Thief', 'C. Tangana',
       'Cazzu', 'Charli XCX', 'Gilles Peterson', 'Jay Electronica',
       'Khruangbin', 'King Gizzard & The Lizzard Wizard', 'Mall Grab',
       'Metronomy', 'Perfume', 'Playboi Carti', 'Slowdive', 'A.G. Cook',
       'Agoraphobia', 'Alex Cameron', 'Ambarchi', 'Sprenger', 'Sollman',
       'Anz', 'Bikoko', 'Casero', 'Cuban Doll', 'Dry Cleaning', 'Duma',
       'El Mato a un policia motorizado', 'Future Utopia',
       'Gabber Modus Operandi', 'Gracey', 'Griff', 'Haai', 'High On Fire',
       'Holly Humberstone', 'India Jordan', 'Jensen Mcrae',
       'John Talabot', 'Moonchild Sanelly', 'Mujeres', 'Pile', 'Prospa',
       'Rata Negra', 'Slikback', 'Sofia Kourtesis', 'Sonida Tupinamba',
       'Squid', 'Tarta Relena', "Working Men's Club", 'Lorde',
       'Massive Attack', 'The Strokes', 'Black Coffee', 'Brittany Howard',
       'Burna Boy', 'Clairo', 'Cou

In [6]:
artists_url = 'https://api.setlist.fm/rest/1.0/search/artists'

In [8]:
lineup_mbids = []
not_found = []

for name in lineup:
    req = requests.get(artists_url,
                       headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                       params={'artistName': name, 'p': 1, 'sort': 'relevance'}
    )
    
    i = 0
    while (not req.ok) & (i <= 5):
        req = requests.get(artists_url,
                           headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                           params={'artistName': name, 'p': 1, 'sort': 'relevance'}
        )
        i += 1
    
    if req.ok:
        artist_response = req.json()['artist']
        num_artists = len(artist_response)
        if num_artists > 1:
            for i in range(num_artists):
                if artist_response[i]['name'].lower() == name.lower():
                    mbid = artist_response[i]['mbid']
                    lineup_mbids.append({'name': name, 'mbid': mbid})
                    break
        elif num_artists == 1:
            mbid = artist_response[0]['mbid']
            lineup_mbids.append({'name': name, 'mbid': mbid})
        elif num_artists == 0:
            print(f'No results I think for {name}')
        else:
            print(f'WTF {name}?')
                    
    else:
        print(f'Couldn\'t find {name}')
        not_found.append(name)

Couldn't find King Gizzard & The Lizzard Wizard
Couldn't find Sollman
Couldn't find Bikoko
Couldn't find Future Utopia
Couldn't find Sonida Tupinamba
Couldn't find Amaarae
Couldn't find Cmat
Couldn't find LCY
Couldn't find Marta Knight
Couldn't find Rigoberta Bandini
Couldn't find Sama Abdulhadi
Couldn't find Tarquim
Couldn't find Acemoma
Couldn't find Confetti de Odio
Couldn't find Paranoid 1996
Couldn't find Unai Mugurza


In [9]:
lineup_mbids

[{'name': 'Dua Lipa', 'mbid': '6f1a58bf-9b1b-49cf-a44a-6cefad7ae04f'},
 {'name': 'Gorillaz', 'mbid': 'e21857d5-3256-4547-afb3-4b6ded592596'},
 {'name': 'Interpol', 'mbid': 'b23e8a63-8f47-4882-b55b-df2c92ef400e'},
 {'name': 'Tyler, The Creator',
  'mbid': 'f6beac20-5dfe-4d1f-ae02-0b0a740aafd6'},
 {'name': 'Ashnikko', 'mbid': '7e3e73e3-41a7-435c-9af3-484c0d3ba682'},
 {'name': 'Bad Gyal', 'mbid': '9cb2e99f-d0ba-4aa5-a371-0006b0d34090'},
 {'name': 'Bicep', 'mbid': '10e8f0df-90f9-4957-aedd-c4e0d187e3a8'},
 {'name': 'Big Thief', 'mbid': '9f81247f-7f57-42f3-a8ba-75bef554e591'},
 {'name': 'C. Tangana', 'mbid': '46d3689c-23e6-46f7-8ad6-c12f13ac9dea'},
 {'name': 'Cazzu', 'mbid': '1d47b224-2cd9-4852-8e55-b9ec635e0c43'},
 {'name': 'Charli XCX', 'mbid': '260b6184-8828-48eb-945c-bc4cb6fc34ca'},
 {'name': 'Gilles Peterson', 'mbid': '4c4e3121-4d12-4f7a-a77c-5becd849fb3c'},
 {'name': 'Jay Electronica', 'mbid': '8bf283b9-b1e2-4632-a953-eacd212a543e'},
 {'name': 'Khruangbin', 'mbid': 'aea4c9b9-9f8d-49dc-

In [10]:
not_found

['King Gizzard & The Lizzard Wizard',
 'Sollman',
 'Bikoko',
 'Future Utopia',
 'Sonida Tupinamba',
 'Amaarae',
 'Cmat',
 'LCY',
 'Marta Knight',
 'Rigoberta Bandini',
 'Sama Abdulhadi',
 'Tarquim',
 'Acemoma',
 'Confetti de Odio',
 'Paranoid 1996',
 'Unai Mugurza']

In [11]:
artist_setlist = []

for a in lineup_mbids:
    songs_played = []
    mbid = a['mbid']
    setlists_url = f'https://api.setlist.fm/rest/1.0/artist/{mbid}/setlists'
    
    req = requests.get(setlists_url,
             headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
             params={'p': 1}
    )
    
    i = 0
    while (not req.ok) & (i <= 5):
        req = requests.get(setlists_url,
                 headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                 params={'p': 1}
        )
        i += 1
            
    if req.ok:
    
        setlist_response = req.json()['setlist']
        num_setlists = len(setlist_response)

        for i in range(num_setlists):
            setlist = setlist_response[i]['sets']['set']
            num_sections = len(setlist)
            total_songs = []
            for p in range(num_sections):
                total_songs += setlist[p]['song']
            num_songs = len(total_songs)

            for i in range(num_songs):
                song = total_songs[i]
                song_title = song['name']
                # if the song is a cover add the original artist to the song title
                if 'cover' in song:
                    song_title += ' {}'.format(song['cover']['name'])
                songs_played.append(song_title)
                                           
        most_played_songs = list(pd.Series(songs_played).value_counts().head(15).index)

        artist_setlist.append({
            'artist': a['name'],
            'setlist': most_played_songs
        })
    else:
        not_found.append(a['name'])

<ipython-input-11-d1dc557b161d>:42: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  most_played_songs = list(pd.Series(songs_played).value_counts().head(15).index)


In [12]:
not_found

['King Gizzard & The Lizzard Wizard',
 'Sollman',
 'Bikoko',
 'Future Utopia',
 'Sonida Tupinamba',
 'Amaarae',
 'Cmat',
 'LCY',
 'Marta Knight',
 'Rigoberta Bandini',
 'Sama Abdulhadi',
 'Tarquim',
 'Acemoma',
 'Confetti de Odio',
 'Paranoid 1996',
 'Unai Mugurza',
 'Anz',
 'India Jordan',
 'Jensen Mcrae',
 'Prospa',
 'Sofia Kourtesis',
 'Tarta Relena',
 'Erika de Casier',
 'Valentino Mora',
 'Angele',
 'Fred Again..',
 'La Mafia del Amor',
 'Tainy',
 'DJ Pastis',
 'Sad Night Dynamite',
 'Sherelle',
 'Tim Reaper']

In [13]:
artist_setlist

[{'artist': 'Dua Lipa',
  'setlist': ['Levitating',
   "Don't Start Now",
   'Pretty Please',
   'Break My Heart',
   'Love Again',
   'Physical',
   'Hallucinate',
   'New Rules',
   'Fever',
   'UN DIA (ONE DAY)',
   "We're Good",
   'Electricity Silk City & Dua Lipa',
   'Club Disco',
   'Dance Break',
   'Cool']},
 {'artist': 'Gorillaz',
  'setlist': ['Last Living Souls',
   'Clint Eastwood',
   'On Melancholy Hill',
   'Humility',
   'M1 A1',
   'El Mañana',
   'Souk Eye',
   'Tranz',
   'Every Planet We Reach Is Dead',
   'Tomorrow Comes Today',
   'Andromeda',
   'Dirty Harry',
   'Feel Good Inc.',
   'Strobelite',
   'Kids With Guns']},
 {'artist': 'Interpol',
  'setlist': ['Rest My Chemistry',
   'Slow Hands',
   'The Rover',
   'The Heinrich Maneuver',
   "C'mere",
   'Evil',
   'If You Really Love Nothing',
   'The New',
   'Untitled',
   'Narc',
   'Obstacle 1',
   'Stay in Touch',
   'NYC',
   'Complications',
   'PDA']},
 {'artist': 'Tyler, The Creator',
  'setlist': ['NE

In [14]:
setlist_lengths = []
short_or_empty_setlist = []

for i in range(len(artist_setlist)):
    n_songs = len(artist_setlist[i]['setlist'])
    setlist_lengths.append({
        'artist': artist_setlist[i]['artist'],
        'n_songs': n_songs
    })
    
    if n_songs < 5:
        short_or_empty_setlist.append(artist_setlist[i]['artist'])

In [15]:
len(short_or_empty_setlist)

40

In [16]:
short_or_empty_setlist

['Mall Grab',
 'A.G. Cook',
 'Cuban Doll',
 'Duma',
 'Gabber Modus Operandi',
 'Griff',
 'Haai',
 'John Talabot',
 'Moonchild Sanelly',
 'Rata Negra',
 'Slikback',
 'Giveon',
 'Khea',
 'Nicola Cruz',
 'Pa Salieu',
 'Alizzz',
 'Efdemin',
 'Marco Shuttle',
 'Maeve',
 'Mary Anne Hobbs',
 'Menta',
 'Oli XL',
 'Pangaea',
 'Priya Ragu',
 'Special Interest',
 'SPFDJ',
 'Vladislav Delay',
 'VTSS',
 'Wata Igarashi',
 'DJ Seinfeld',
 'Romy',
 'Angel Bat Dawid',
 'Arooj Aftab',
 'Courtesy',
 'Craig Richards',
 'Nicolas Lutz',
 'Goldie',
 'Logic1000',
 'Namasenda',
 'Special Request']

### Spotify

In [17]:
username = 'adrialuz'
scope = 'playlist-modify-public'

token = spotipy.util.prompt_for_user_token(username, scope)

sp = spotipy.Spotify(auth=token)

sp.trace = False

Couldn't read cache at: .cache-adrialuz
Couldn't read cache at: .cache-adrialuz


In [20]:
sp.search('artist:Dua Lipa', limit=1, type='artist', market='GB')['artists']['items'][0]['id']

'6M2wZ9GZgrQXHCFfjv46we'

In [21]:
sp.search(
        f'artist:Khaled', limit=2, type='artist', market='GB'
    )['artists']['items']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/0QHgL1lAIqAw0HtD7YldmP'},
  'followers': {'href': None, 'total': 6085356},
  'genres': ['hip hop',
   'miami hip hop',
   'pop',
   'pop rap',
   'rap',
   'southern hip hop',
   'trap'],
  'href': 'https://api.spotify.com/v1/artists/0QHgL1lAIqAw0HtD7YldmP',
  'id': '0QHgL1lAIqAw0HtD7YldmP',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab6761610000e5ebbf921114b7f19be97ce29647',
    'width': 640},
   {'height': 320,
    'url': 'https://i.scdn.co/image/ab67616100005174bf921114b7f19be97ce29647',
    'width': 320},
   {'height': 160,
    'url': 'https://i.scdn.co/image/ab6761610000f178bf921114b7f19be97ce29647',
    'width': 160}],
  'name': 'DJ Khaled',
  'popularity': 88,
  'type': 'artist',
  'uri': 'spotify:artist:0QHgL1lAIqAw0HtD7YldmP'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/28ztjHIXceRRntmTUfnmUX'},
  'followers': {'href': None, 'total': 1183877},
  'genres': ['classic arab p

In [22]:
spotify_ids = []
for a in short_or_empty_setlist:
    search_result = sp.search(
        f'artist:{a}', limit=5, type='artist', market='GB'
    )['artists']['items']
    
    if search_result:
        for i in range(len(search_result)):
            name = search_result[i]['name']
            if name.lower() == a.lower():
                artist_id = search_result[i]['id']
                spotify_ids.append(artist_id)
                break
            else:
                pass
    else:
        print(f'Couldn\'t find {a} on Spotify.')

Couldn't find Mary Anne Hobbs on Spotify.
Couldn't find SPFDJ on Spotify.
Couldn't find Nicolas Lutz on Spotify.


In [23]:
spotify_ids

['7yF6JnFPDzgml2Ytkyl5D7',
 '0c6t5Anm4ZEH2sAaPJH2yU',
 '32DuKytBMZn90iqyh2HEdc',
 '4z8y2MjTFwLa73dABYP1io',
 '5RJFJWYgtgWktosLrUDzff',
 '0pkLgeB9j465x1QB2kRoy4',
 '1YvN5uOGQkHVUUlZUcnotD',
 '6aDX1jzNVAI9enlQzW0Pgw',
 '7KjdFWj7ujhSLbzWYAAgVe',
 '0NwRAG9DawUqqgur9925fA',
 '4fxd5Ee7UefO4CUXgwJ7IP',
 '4m6ubhNsdwF4psNf3R8kwR',
 '0OltT51j3hIkgaDJqqPzDn',
 '290nCNEce1y6rfoJiO2rK7',
 '23herDudxPBB3S81GB5uG3',
 '6hjRjVNLWTCPYci9nxhI1G',
 '2DqPerh6whhlts8Pb0BUJi',
 '0tjJosKJDhjhGh2P8NtiK3',
 '3L3ejltt5dmjXkES4YSGKX',
 '08Z0yhWGksNk3wceqlCeGE',
 '6iZTyHbQWGzpiWoyI0zz9F',
 '2CYTLJOt91YLe1JLStFu6m',
 '5E3H2KyR31E2Dj3K6vIUe9',
 '0zo109NM3S7CqHpvlXwqEN',
 '7ug2B8FOnKHqwtVlD9vrQX',
 '37YzpfBeFju8QRZ3g0Ha1Q',
 '3X2DdnmoANw8Rg8luHyZQb',
 '59lypjMPo7ZILhNMCgaE19',
 '00JAfwtx5gNiiqyor88Dr5',
 '1pRmI23pmMBJU04aalTI6M',
 '6EVc5gJQXiUl586m1NFCNH',
 '2SYqJ3uDLLXZNyZdLKBy4M',
 '2EFsfh1zewsSWhDINv7j1I',
 '5T68nryXXOMNE2kVe61fKX',
 '59xdAObFYuaKO2phzzz07H']

In [25]:
sp.artist('59xdAObFYuaKO2phzzz07H')['name']

'Special Request'

In [26]:
popular_songs = []

for artist_id in spotify_ids:
    search_results = sp.artist_top_tracks(artist_id, country='GB')['tracks']

    top_songs = []
    if search_results:
        for i in range(len(search_results)):
            song_name = search_results[i]['name']
            top_songs.append(song_name)
        popular_songs.append({
            'artist': sp.artist(artist_id)['name'],
            'setlist': top_songs
        })
    else:
        print(artist_id, sp.artist(artist_id)['name'])

In [27]:
popular_songs

[{'artist': 'Mall Grab',
  'setlist': ['Liverpool Street In The Rain',
   'Pool Party Music',
   "I've Always Liked Grime",
   'FMG',
   'Menace II Society',
   'Sunflower',
   'Catching Feelings - Edit',
   'Strangers',
   'Leaving Tokyo',
   'Room Full of Rothko - Edit']},
 {'artist': 'A.G. Cook',
  'setlist': ['Doing It (feat. Rita Ora) - A.G. Cook Remix']},
 {'artist': 'Cuban Doll',
  'setlist': ['Bankrupt',
   'Bankrupt (Feat. Lil Yachty & Lil Baby)',
   'Let It Blow (feat. Molly Brazy)',
   "Ain't Bout It",
   'Drug Dealer',
   'My Nigga 2',
   'Ohchee Walla',
   '1st Off',
   'Fuck Love',
   'Mo Money']},
 {'artist': 'Gabber Modus Operandi',
  'setlist': ['Sangkakala',
   'Hey Nafsu',
   'Kon',
   'Semeton 10 Ton',
   'Sangkakala III',
   'Trance Adiluhunxxx',
   'Sangkakala',
   'Nyonyon Rangda',
   'Melayu Boleh Neraka - Exclusive Bonus',
   'Balap Liar - Exclusive Bonus']},
 {'artist': 'Griff',
  'setlist': ['Black Hole',
   'Inside Out (feat. Griff)',
   'Good Stuff',
   '1,

Get the URI codes for each track

In [28]:
uris = []
missing_songs = []

for a in (artist_setlist + popular_songs):
    artist = a['artist']
    setlist = a['setlist']
    for s in setlist:
        s = s.replace(',', '').replace('\'', '').replace('"', '').replace('.', '').replace(
            '?', '').replace(')', '').replace('(', '').replace('/', '').replace(
            '\\', '').replace('&', '').replace('-', '')
        items = sp.search(q=f'artist:{artist} track:{s}', limit=1)['tracks']['items']
        if items:
            uri = items[0]['id']
            uris.append(uri)
        else:
            items = sp.search(q=f'track:{s}', limit=1)['tracks']['items']
            if items:
                if items != [None]:
                    uri = items[0]['id']
                    uris.append(uri)
            else:
                missing_songs.append({
                    'artist': artist,
                    'song': s
                })

In [29]:
len(uris)

1371

In [30]:
len(missing_songs)

75

In [31]:
missing_songs

[{'artist': 'Cazzu', 'song': 'C14ORCE II'},
 {'artist': 'Gilles Peterson', 'song': 'Floating Points – Karakul'},
 {'artist': 'Gilles Peterson', 'song': 'Miles Davis   Maze Warner'},
 {'artist': 'Gilles Peterson', 'song': 'Floating Points – Environments'},
 {'artist': 'Gilles Peterson', 'song': 'Headie One  Skepta – Back To Basics'},
 {'artist': 'Gilles Peterson', 'song': 'Tommy Smith – Easter Island'},
 {'artist': 'Gilles Peterson',
  'song': 'Esnard Boisdur   Mizik Bel A808 RemixFavorite France'},
 {'artist': 'Gilles Peterson', 'song': 'Anunnaku   Forgotten Tales Whities'},
 {'artist': 'Gilles Peterson', 'song': 'TSVI – Labryinth'},
 {'artist': 'Gilles Peterson', 'song': 'Floating Points – Les Alpx'},
 {'artist': 'Gilles Peterson', 'song': 'Ti Seles – Pa Dekourajew Toni'},
 {'artist': 'Gilles Peterson', 'song': 'Lucked In – Nuttin Cold'},
 {'artist': 'Gilles Peterson',
  'song': 'Hermeto Pascoal   Santa Antonio Warner Bros Records'},
 {'artist': 'Gilles Peterson',
  'song': 'James Bra

In [32]:
divisor = int(np.floor(len(uris) / np.ceil(len(uris) / 100)))
times = int(np.floor(len(uris) / divisor))

In [33]:
for i in range(times):
    subset = uris[divisor*i:divisor*(i+1)]
    sp.user_playlist_add_tracks(username, playlist_id='5sfuRJ3UgOv35RizTPycAC',
                                tracks=subset)